## Setup
Using pytorch and pennylane to construct the novel QNN.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

try:
   import pennylane as qml
except ImportError:
  !pip install pennylane
  import pennylane as qml

import numpy as np
import matplotlib.pyplot as plt

kernel_size = 5
images = 1
alpha = 0.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 87.2 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


## Classical CNN Layers


In [ ]:
### Convolution 1
C1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2)
x = torch.rand(images, 1, 28, 28)
# activations = nn.ReLU()
# activations = nn.Sigmoid()
# activations = nn.Tanh()
x1 = C1(x)
print("C1 Shape:", x1.shape)

### Pooling 1
P1 = nn.AvgPool2d(kernel_size=2, stride=2)
p1 = P1(x1)
print("P1 Shape:", p1.shape)

### Convolution 2
C2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
x2 = C2(p1)
print("C2 Shape:", x2.shape)

### Pooling 2
P2 = nn.AvgPool2d(kernel_size=1, stride=1)
p2 = P2(x2)
print("P2 Shape:", p2.shape)

### Full connections
FC1 = nn.Linear(in_features=16*10*10, out_features=120)
x3 = FC1(p2.view(-1, 16*10*10))
print("F2 Shape", x3.shape)

FC2 = nn.Linear(in_features=120, out_features=64)
x4 = FC2(x3)
print("F3 Shape",x4.shape)

C1 Shape: torch.Size([1, 6, 28, 28])
P1 Shape: torch.Size([1, 6, 14, 14])
C2 Shape: torch.Size([1, 16, 10, 10])
P2 Shape: torch.Size([1, 16, 10, 10])
F2 Shape torch.Size([1, 120])
F3 Shape torch.Size([1, 64])


##Residual Full Connection

In [ ]:
rfc = nn.Linear(in_features=64, out_features=10)
rfc_out = rfc(x4)

print("F_residual Shape:", rfc_out.shape)
print(rfc_out)

F_residual Shape: torch.Size([1, 10])
tensor([[-0.0176, -0.1110, -0.1148, -0.1015, -0.0327,  0.1289, -0.0133, -0.0727,
          0.0326,  0.1240]], grad_fn=<AddmmBackward0>)


##QNN

In [ ]:
#QNN Setup

def init_random_hermitian_observable(n_qubits):
  Hermitian = torch.randn(n_qubits, 4)
  return Hermitian

In [ ]:
### Actual Quantum Layer - Kris
n_qubits = 6 #2^6 = 64, so will embed the length 64 vector
entangling_layers = 3 # The PQC strongly entangling layers happens 3 times.
fixed_hermitians = True

dev = qml.device("default.qubit", wires=n_qubits)
I = torch.tensor([[1,0],[0,1]])
X = torch.tensor([[0,1],[1,0]])
Y = torch.tensor([[0,-1j],[1j,0]])
Z = torch.tensor([[1,0],[0,-1]])

# Define Quantum Circuit and Link with PyTorch
@qml.qnode(dev, interface='torch')
def qnn_circuit(inputs, weights, hermitians):
  # Amplitude Embedding
  qml.AmplitudeEmbedding(inputs, wires=range(n_qubits), normalize=True)

  # Strong Entangling Layers
  qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))

  # Measure Hermitian Observable

  expvals = []

  for q in range(n_qubits):
    ai, ax, ay, az = hermitians[q]
    observable = (ai*I) + (ax*X) + (ay*Y) + (az*Z)

    expvals.append(qml.expval(qml.Hermitian(observable, q)))

  return expvals

# Define weights and have 3 layers
# This defines 3 repetitions of the entangling layers on the 6 qubits, with 3 weights
weight_shapes = {
    "weights":(entangling_layers, n_qubits, 3),
    "hermitians":(n_qubits, 4)
}

# Creates the quantum layer
qnn_layer = qml.qnn.TorchLayer(qnn_circuit, weight_shapes)

#Initialize Hermitians randomly
with torch.no_grad():
    hermitian_init = init_random_hermitian_observable(n_qubits)
    qnn_layer.hermitians.copy_(hermitian_init)

if(fixed_hermitians):
  qnn_layer.hermitians.requires_grad_(False)

# Passes the 64 inputs into the quantum layer
qnn = qnn_layer(x4)
print("Quantum Layer Shape:", qnn.shape)

# Final qnn classification layer, converts the 6 qubits to 10 outputs
qnnfcl = nn.Linear(in_features=6, out_features=10)
qnn_out = qnnfcl(qnn)
print("Final Quantum Layer Shape:",qnn_out.shape)
print(qml.draw(qnn_circuit, level='device')(x4[0], qnn_layer.weights, qnn_layer.hermitians))

Quantum Layer Shape: torch.Size([1, 6])
Final Quantum Layer Shape: torch.Size([1, 10])
0: ─╭|Ψ⟩──Rot(0.58,6.15,4.68)─╭●─────────────╭X──Rot(4.74,6.20,3.70)─╭●──────────╭X ···
1: ─├|Ψ⟩──Rot(5.15,2.30,4.49)─╰X─╭●──────────│───Rot(4.89,5.18,5.37)─│──╭●───────│─ ···
2: ─├|Ψ⟩──Rot(5.79,3.92,1.39)────╰X─╭●───────│───Rot(1.75,3.50,2.76)─╰X─│──╭●────│─ ···
3: ─├|Ψ⟩──Rot(0.54,4.89,0.10)───────╰X─╭●────│───Rot(0.70,0.94,2.98)────╰X─│──╭●─│─ ···
4: ─├|Ψ⟩──Rot(5.38,2.34,0.29)──────────╰X─╭●─│───Rot(6.20,1.76,3.23)───────╰X─│──╰● ···
5: ─╰|Ψ⟩──Rot(4.43,3.71,5.75)─────────────╰X─╰●──Rot(4.15,4.06,2.69)──────────╰X─── ···

0: ··· ──Rot(5.21,3.24,5.06)──────────────────────╭●───────╭X───────┤  <𝓗(M0)>
1: ··· ─╭X────────────────────Rot(6.22,5.55,6.09)─│──╭●────│──╭X────┤  <𝓗(M1)>
2: ··· ─│─────────────────────Rot(4.47,0.65,1.24)─│──│──╭●─│──│──╭X─┤  <𝓗(M2)>
3: ··· ─│─────────────────────Rot(0.36,1.48,5.22)─╰X─│──│──╰●─│──│──┤  <𝓗(M3)>
4: ··· ─│─────────────────────Rot(0.91,5.71,4.29)────╰X─│─────╰●─│──

##Output

In [ ]:
# Combines the two outputs using a sum based on alpha
out = (rfc_out * (1-alpha)) + (qnn_out * alpha)

print(out)
print(out.shape)

tensor([[-0.5556, -0.4546, -0.5087,  0.0859,  0.0914,  0.4580,  0.5325,  0.3393,
          0.3355,  0.4236]], grad_fn=<AddBackward0>)
torch.Size([1, 10])
